<a href="https://colab.research.google.com/github/SammyDMartin/CC/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
#Uncomment for Colab
use_cc = False
"""
!pip install "ray[rllib]" torch
!pip install nashpy
!pip install pickle5
!git clone https://github.com/SammyDMartin/CC.git
!cd CC

use_CC = True
"""

'\n!pip install "ray[rllib]" torch\n!pip install nashpy\n!pip install pickle5\n!git clone https://github.com/SammyDMartin/CC.git\n!cd CC\n\nuse_CC = True\n'

In [45]:
if use_cc == True:
    from CC.cc_search import assess_agent_vs_others,assess_agent_vs_agent
    from CC.cc_tools import DQNTorchPolicy,DQNTrainer,PPOTorchPolicy,PPOTrainer,A3CTorchPolicy,A3CTrainer,BRToLastPolicy,IteratedChicken
else:
    from cc_search import assess_agent_vs_others,assess_agent_vs_agent
    from cc_tools import DQNTorchPolicy,DQNTrainer,PPOTorchPolicy,PPOTrainer,A3CTorchPolicy,A3CTrainer,BRToLastPolicy,IteratedChicken,CoopPolicy,DefectPolicy,RandPolicy,MaximinBullyPolicy

In [46]:
opponents = [(DQNTorchPolicy,DQNTrainer),(PPOTorchPolicy,PPOTrainer),(A3CTorchPolicy,A3CTrainer)]
strategies = [BRToLastPolicy]

In [47]:
import warnings
warnings.filterwarnings("ignore")

In [48]:
IteratedChicken.PAYOUT_MATRIX

array([[[  0.,   0.],
        [ -1.,   1.]],

       [[  1.,  -1.],
        [-10., -10.]]])

In [49]:
#res1, res_full = assess_agent_vs_others(agent_trainer=(BRToLastPolicy,None),opponents=[], strategies=[BRToLastPolicy],steps=2)

exanal = assess_agent_vs_agent(policies=[BRToLastPolicy,MaximinBullyPolicy],checkpoints=[],environment=IteratedChicken,iters=5)

 pid=10600) 2022-01-26 20:49:39,411	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
 pid=10600) 2022-01-26 20:49:39,419	WARNING util.py:57 -- Install gputil for GPU system monitoring.
 pid=10600) 2022-01-26 20:49:39,424	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!


 pid=10600) ray version: 1.9.2


In [50]:
exanal.trial_dataframes[list(exanal.trial_dataframes.keys())[0]]

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,timesteps_this_iter,agent_timesteps_total,done,...,timers/learn_time_ms,timers/learn_throughput,info/num_steps_sampled,info/num_agent_steps_sampled,info/num_steps_trained,info/num_agent_steps_trained,info/last_target_update_ts,info/num_target_updates,perf/cpu_util_percent,perf/ram_util_percent
0,0.0,-20.0,-7.8,10.0,100,0,1000,0,2000,False,...,3.012,10623.534,1000,2000,32,64,1000,1,46.000000,81.100
1,0.0,-20.0,-10.0,10.0,100,0,2000,0,4000,False,...,0.000,0.000,2000,4000,8032,16064,1504,2,31.950000,81.200
2,0.0,-20.0,-11.8,10.0,100,0,3000,0,6000,False,...,0.000,0.000,3000,6000,16032,32064,2512,4,33.866667,81.200
3,0.0,-20.0,-10.0,10.0,100,0,4000,0,8000,False,...,0.000,0.000,4000,8000,24032,48064,3520,6,35.075000,79.975
4,0.0,-20.0,-8.2,10.0,100,0,5000,0,10000,True,...,0.000,0.000,5000,10000,32032,64064,4528,8,31.800000,79.850
